<h1 style="text-align:center;">Computational Intelligence</h1>
<h2 style="text-align:center;">Classification with Convolutional Neural Network (CNN)</h2>
<h4 style="text-align:center;">by H. Naderan</h4>
<h5 style="text-align:center;">
Mechanical Engineering Department<br>
Amirkabir University of Technology
</h5>

***

### Preliminaries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img = plt.imread("Bikesgray.png")

In [ ]:
img.shape

In [ ]:
plt.imshow(img, cmap = "Greys_r")

### Defining a function for filter apply

In [ ]:
def apply_filter(image, filter):
    h, w = image.shape

    newimage = np.zeros((h - 2, w - 2))
    for i in range(h - 2):
        for j in range(w - 2):
            newimage[i, j] = np.sum(image[i:(i + 3), j:(j + 3)]*filter, axis=(0, 1))
    
    return newimage

### Sobel filters

In [ ]:
sobelv = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]])
sobelh = np.array([[ 1,  2,  1],
                   [ 0,  0,  0],
                   [-1, -2, -1]])

In [ ]:
imgv = apply_filter(img, sobelv)
imgh = apply_filter(img, sobelh)

In [ ]:
plt.figure(figsize=(10, 20))
plt.subplot(1, 2, 1)
plt.imshow(imgv, cmap="Greys_r")
plt.subplot(1, 2, 2)
plt.imshow(imgh, cmap="Greys_r")

### Preparing data

In [ ]:
import pandas as pd

In [ ]:
trainData = pd.read_csv("mnist_train.zip")
testData = pd.read_csv("mnist_test.zip")
Xtrain, Ytrain = trainData.iloc[:, 1:].to_numpy(), trainData.iloc[:, 0].to_numpy()
Xtest, Ytest = testData.iloc[:, 1:].to_numpy(), testData.iloc[:, 0].to_numpy()
Xtrain, Xtest = Xtrain.reshape(-1, 28, 28), Xtest.reshape(-1, 28, 28)

In [ ]:
Xtrain = Xtrain.astype("float32")/255
Xtest = Xtest.astype("float32")/255

In [ ]:
Ytrain = keras.utils.to_categorical(Ytrain)
Ytest  = keras.utils.to_categorical(Ytest)

In [ ]:
Xtrain.shape

In [ ]:
plt.figure(figsize=(18, 5))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.axis(True)
    plt.imshow(Xtrain[i], cmap='gray')
    plt.subplots_adjust(wspace=0.2, hspace=0.2)

### Building a Convolutional Neural Network

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras_core as keras
from keras_core.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten


num_filters = 8
filter_size = 3
pool_size = 2

model = keras.Sequential([
    Input(shape=(*Xtrain.shape[1:], 1)),
    #keras.layers.Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
    Conv2D(num_filters, filter_size),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
    Dense(10, activation='softmax'),
])
model.summary()

In [ ]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.CategoricalAccuracy(name="acc"),
    ],
)

### Training the model

In [ ]:
batch_size = 128
epochs = 20

In [ ]:
trainState = model.fit(
    Xtrain,
    Ytrain,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
)

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)

def plot_results(metrics, title=None, ylabel=None, ylim=None, metric_name=None, color=None):

    fig, ax = plt.subplots(figsize=(15, 4))

    if not (isinstance(metric_name, list) or isinstance(metric_name, tuple)):
        metrics = [metrics,]
        metric_name = [metric_name,]

    for idx, metric in enumerate(metrics):    
        ax.plot(metric, color=color[idx])

    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.xlim([0, 20])
    plt.ylim(ylim)
    # Tailor x-axis tick marks
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    plt.grid(True)
    plt.legend(metric_name)   
    plt.show()
    plt.close()

In [ ]:
trainLoss = trainState.history["loss"]
trainAcc  = trainState.history["acc"]
validLoss = trainState.history["val_loss"]
validAcc  = trainState.history["val_acc"]

plot_results([trainLoss, validLoss],        
            ylabel="Loss", 
            ylim = [0.0, 0.5],
            metric_name=["Training Loss", "Validation Loss"],
            color=["g", "b"]);

plot_results([trainAcc, validAcc], 
            ylabel="Accuracy",
            ylim = [0.9, 1.0],
            metric_name=["Training Accuracy", "Validation Accuracy"],
            color=["g", "b"])

In [ ]:
w = model.layers[0].get_weights()[0]

In [ ]:
w.shape

In [ ]:
w[:,:, 0, 0]

In [ ]:
plt.imshow(w[:, :, 0, 0], cmap="gray")

In [ ]:
fig, ax = plt.subplots(5, 8)
for j in range(5):
    for i in range(8):
        ax[j, i].imshow(apply_filter(Xtrain[j, :, :], w[:, :, 0, i]), cmap="Greys_r")